In [4]:
import cv2
import mediapipe as mp
import os
import pandas as pd

mp_pose = mp.solutions.pose

def extract_landmarks_from_video(video_path):
    pose = mp_pose.Pose(static_image_mode=False)
    cap = cv2.VideoCapture(video_path)

    data = []
    frames_con_landmarks = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(frame_rgb)

        if results.pose_landmarks:
            frames_con_landmarks += 1
            frame_landmarks = []
            for lm in results.pose_landmarks.landmark:
                frame_landmarks.extend([lm.x, lm.y, lm.z, lm.visibility])
            data.append(frame_landmarks)

    cap.release()
    pose.close()

    print(f"📹 {os.path.basename(video_path)} → {frames_con_landmarks} frames con landmarks.")
    return pd.DataFrame(data)

def process_all_videos(input_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    processed_files = set()
    for f in os.listdir(output_dir):
        if f.endswith('.csv'):
            processed_files.add(f.replace('.csv', ''))
    print(f"🔍 Archivos CSV procesados previamente: {(processed_files)}")

    for root, _, files in os.walk(input_dir):
        activity = os.path.basename(root)
        for file in files:
            if file.endswith(".mp4"):
                output_name = f"{activity}_{file.replace('.mp4', '')}"
                if output_name in processed_files:
                    print(f"⏩ Saltando {file}, ya procesado.")
                    continue
                full_path = os.path.join(root, file)
                output_path = os.path.join(output_dir, f"{output_name}.csv")
                print(f"🚀 Procesando: {file}")
                df = extract_landmarks_from_video(full_path)
                if not df.empty:
                    df.to_csv(output_path, index=False)
                    print(f"✅ Guardado en: {output_path}")
                else:
                    print(f"⚠️ No se detectaron landmarks en {file}, CSV no generado.")

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
input_dir = os.path.join(project_root, "data", "raw_videos")
output_dir = os.path.join(project_root, "data", "processed_landmarks")

# Ejecutar
process_all_videos(input_dir, output_dir)


🔍 Archivos CSV procesados previamente: {'caminar_frente_user1_frente_back_3', 'levantarse_user1_levantarse_back_1', 'girar_user1_girar_left_2', 'levantarse_user1_levantarse_back_2', 'levantarse_user1_levantarse_right_1', 'levantarse_user3_levantarse_front2', 'levantarse_user2_levantarse_front_2', 'levantarse_user1_levantarse_left_1', 'levantarse_user1_levantarse_left_3', 'levantarse_user3_levantarse_front3', 'caminar_lado_user2_side_left', 'caminar_lado_user2_side_right', 'caminar_frente_user1_frente_back_1', 'girar_user1_gitar_right_1', 'levantarse_user3_levantarse_front1', 'girar_user1_girar_left_1', 'sentarse_user2_sentarse_front_2', 'sentarse_user2_sentarse_left', 'girar_user2_girar_right', 'levantarse_user1_levantarse_left_2', 'caminar_espalda_user1_espalda_front_2', 'levantarse_user2_levantarse_back', 'saltar_user2_saltar_front_1', 'sentarse_user1_sentarse_left_1', 'sentarse_user1_sentarse_front_2', 'caminar_espalda_user1_espalda_back_1', 'caminar_espalda_user2_espalda_front', 'l

In [5]:
# Mostrar el directorio de trabajo actual para depuración
import os
print('Directorio de trabajo actual:', os.getcwd())

Directorio de trabajo actual: c:\Users\CTecn\Desktop\Inteligencia-Artificial\notebooks


In [6]:
# Extraer características (ejemplo: ángulo de cadera) y crear feature_data.csv
import os
import pandas as pd
import numpy as np
def calcular_angulo_3p(p1, p2, p3):
    # Calcula el ángulo entre tres puntos en 3D
    a = np.array(p1)
    b = np.array(p2)
    c = np.array(p3)
    ba = a - b
    bc = c - b
    cos_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cos_angle, -1.0, 1.0))
    return np.degrees(angle)

# Usar rutas relativas al proyecto
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
landmarks_dir = os.path.join(project_root, "data", "processed_landmarks")
annotated_dir = os.path.join(project_root, "data", "annotated_data")
os.makedirs(annotated_dir, exist_ok=True)

salida = []
for file in os.listdir(landmarks_dir):
    if file.endswith(".csv"):
        path = os.path.join(landmarks_dir, file)
        df = pd.read_csv(path)
        if df.empty:
            continue
        # MediaPipe Pose: cadera izq=23, cadera der=24, hombro izq=11, hombro der=12, rodilla izq=25, rodilla der=26
        # Usamos cadera izq (23), cadera der (24), hombro der (12) para ejemplo
        try:
            # Tomar el primer frame del archivo
            row = df.iloc[0]
            # Cada landmark tiene 4 valores (x, y, z, visibility)
            lmk = row.values.reshape(-1, 4)
            cadera_izq = lmk[23][:3]
            cadera_der = lmk[24][:3]
            hombro_der = lmk[12][:3]
            angulo_cadera = calcular_angulo_3p(cadera_izq, cadera_der, hombro_der)
        except Exception as e:
            print(f"Error procesando {file}: {e}")
            continue
        # Extraer label de la actividad del nombre del archivo
        label = file.split('_')[0]
        salida.append({"angle_hip": angulo_cadera, "label": label})

# Guardar el archivo de features
feature_path = os.path.join(annotated_dir, "feature_data.csv")
pd.DataFrame(salida).to_csv(feature_path, index=False)
print("✅ Archivo feature_data.csv generado con", len(salida), "ejemplos.")

✅ Archivo feature_data.csv generado con 101 ejemplos.
